Open AI SDK with MLFLOW

In [0]:
!pip install openai-agents openai mlflow dotenv

In [0]:
dbutils.library.restartPython()

In [0]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel,set_tracing_export_api_key
from agents.run import RunConfig
import mlflow
import os
from IPython.display import display, Markdown
import mlflow

#userdata.get('secretName')

In [0]:


load_dotenv(override=True)
mlflow.login()
# This will create a new experiment called "Tracing Quickstart" and set it as active
mlflow.set_experiment("/Workspace/Users/ichatt82@gmail.com/mlruns/agentsSDK")
mlflow.openai.autolog()
#mlflow.set_experiment(experiment_id=1234)


In [0]:

@mlflow.trace
def external_api():
    # Create an asynchronous OpenAI-style client for calling external APIs (e.g., Gemini)
    external_client = AsyncOpenAI(
        api_key=os.getenv("GOOGLE_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    )

    # Define the model wrapper that communicates with the LLM using OpenAI-compatible schema
    model = OpenAIChatCompletionsModel(
        model="gemini-2.5-flash",
        openai_client=external_client
    )

    # Runtime execution configuration (used to attach model, disable tracing, etc.)
    config = RunConfig(
        model=model,
        model_provider=external_client,
        tracing_disabled=False
    )
    return external_client,model,config

external_client,model,config=external_api()

In [0]:


agent=Agent(name="Agent1",instructions="you are An AI Assisstant",model=model)
# Run the agent asynchronously using Runner with the given input and config
result = await Runner.run(agent, "Tell the latest news with date", run_config=config)
await external_client.close()

In [0]:
display(Markdown(result.final_output))
